<a href="https://colab.research.google.com/github/Lossophy/BINA-Projekt/blob/main/Frage%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q xlrd
!git clone https://github.com/Lossophy/BINA-Projekt.git
!pip install jupyter_bokeh

fatal: destination path 'BINA-Projekt' already exists and is not an empty directory.


#Die Energieversorgung der Schweiz ist in hohem Masse von Importen abhängig.

##Kontext: Im Folgenden wird unsere Rolle im Bundesamt für Energie (BFE) erläutert.

Wir sind in der Abteilung "Betriebswirtschaft und Organisation" des Bundesamts für Energie (BFE) tätig. Im Rahmen unserer Tätigkeit im Kompetenzzentrum für Geoinformation, Open Government Data und Digitalisierung analysieren wir Energiedaten im Kontext der Energiestrategie 2050.

Unser Ziel ist es, historische und aktuelle Daten zur Energiewirtschaft für Politik, Wirtschaft und Öffentlichkeit zugänglich und verständlich aufzubereiten, um fundierte Entscheidungen zu ermöglichen.

---

## Fragestellungen

Die vorliegende Analyse hat zum Ziel, folgende Frage zu beantworten:

* Welche importierten Energieträger haben über die Zeit an Bedeutung gewonnen oder verloren?

---

In [15]:
import pandas as pd

# Datei laden
df = pd.read_csv("/content/BINA-Projekt/Data/ogd115_gest_bilanz.csv")
df.head(170)

,Jahr,Rubrik,Energietraeger,TJ
0,1980,Bruttoverbrauch,Elektrizität,-29450.0
1,1980,Bruttoverbrauch,Erdölprodukte,321250.0
2,1980,Bruttoverbrauch,Fernwärme,0.0
3,1980,Bruttoverbrauch,Gas,36280.0
4,1980,Bruttoverbrauch,Holzenergie,26280.0
...,...,...,...,...
165,1980,Import,Elektrizität,35810.0
166,1980,Import,Erdölprodukte,338250.0
167,1980,Import,Fernwärme,NaN
168,1980,Import,Gas,36860.0


In [ ]:
print(df["Rubrik"].unique())

['Bruttoverbrauch'
 'Eigenverbrauch des Energiesektors, Netzverluste, Verbrauch der Speicherungen'
 'Endverbrauch - Dienstleistungen' 'Endverbrauch - Haushalte'
 'Endverbrauch - Industrie'
 'Endverbrauch - Statistische Differenz inkl. Landwirtschaft'
 'Endverbrauch - Total' 'Endverbrauch - Verkehr'
 'Energieumwandlung - Diverse Erneuerbare' 'Energieumwandlung - Gaswerke'
 'Energieumwandlung - Kernkraftwerke' 'Energieumwandlung - Raffinerien'
 'Energieumwandlung - Wasserkraftwerke'
 'Energieumwandlung - konventionell-thermische Kraft-, Fernheiz- und Fernheizkraftwerke'
 'Export' 'Import' 'Inlandproduktion' 'Lagerveränderung'
 'Nichtenergetischer Verbrauch']


In [22]:
# Typen konvertieren
df["Jahr"] = df["Jahr"].astype(int)
df["TJ"] = df["TJ"].astype(float)

# Filter auf Bruttoverbrauch und importierte Energieträger
df = df[df["Rubrik"] == "Import"]
importierte = ["Elektrizität", "Erdölprodukte","Gas","Kernbrennstoffe","Kohle","Rohöl","Uebrige erneuerbare Energien"]
df = df[df["Energietraeger"].isin(importierte)]


In [ ]:
import plotly.express as px

fig = px.line(
    df,
    x="Jahr",
    y="TJ",
    color="Energietraeger",
    title="Bruttoverbrauch importierter Energieträger in der Schweiz (1980–2023)",
    labels={"TJ": "Energieverbrauch (TJ)", "Energietraeger": "Energieträger"},
    markers=True
)

fig.update_layout(template="plotly_white")
# Alle wichtigen energiepolitischen/geopolitischen Ereignisse (1980–2023)
ereignisse = [
    {"jahr": 1986, "text": "Tschernobyl / AKW-Debatte", "y": 98, "ay": -40},
    {"jahr": 1990, "text": "Kernenergie-Moratorium", "y": 92, "ay": 40},
    {"jahr": 1998, "text": "Strommarkt-Liberalisierung EU", "y": 88, "ay": -40},
    {"jahr": 2000, "text": "1. CO₂-Gesetz Schweiz", "y": 83, "ay": 40},
    {"jahr": 2005, "text": "Beginn Rückgang Erdöl", "y": 78, "ay": -40},
    {"jahr": 2010, "text": "Kohle-Rückgang", "y": 73, "ay": 40},
    {"jahr": 2011, "text": "Fukushima & Atomausstieg", "y": 68, "ay": -40},
    {"jahr": 2017, "text": "Energiestrategie 2050", "y": 63, "ay": 40},
    {"jahr": 2021, "text": "Blockiertes Stromabkommen", "y": 58, "ay": -40},
    {"jahr": 2022, "text": "Ukraine-Krieg / Gas-Krise", "y": 53, "ay": 60},
    {"jahr": 2023, "text": "Winterstromlücke (Debatte)", "y": 120, "ay": -60},
]

# Annotationen & Linien einfügen
for e in ereignisse:
    fig.add_vline(
        x=e["jahr"],
        line_width=2,
        line_dash="dash",
        line_color="gray"
    )
    fig.add_annotation(
        x=e["jahr"],
        y=e["y"],
        text=e["text"],
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        ax=0,
        ay=e["ay"],
        bgcolor="white",
        bordercolor="gray",
        borderwidth=1,
        font=dict(size=11)
    )


fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Daten vorbereiten
df_grouped = df.groupby(["Jahr", "Energietraeger"]).sum().reset_index()
df_total = df_grouped.groupby("Jahr")["TJ"].sum().reset_index().rename(columns={"TJ": "Total"})
df_merged = df_grouped.merge(df_total, on="Jahr")
df_merged["Anteil (%)"] = df_merged["TJ"] / df_merged["Total"] * 100

# Plot erstellen
fig = px.area(
    df_merged,
    x="Jahr",
    y="Anteil (%)",
    color="Energietraeger",
    title="Relative Marktanteile importierter Energieträger (1980–2023)",
    groupnorm="percent"
)

fig.update_layout(template="plotly_white")

# Alle wichtigen energiepolitischen/geopolitischen Ereignisse (1980–2023)
ereignisse = [
    {"jahr": 1986, "text": "Tschernobyl / AKW-Debatte", "y": 98, "ay": -40},
    {"jahr": 1990, "text": "Kernenergie-Moratorium", "y": 92, "ay": 40},
    {"jahr": 1998, "text": "Strommarkt-Liberalisierung EU", "y": 88, "ay": -40},
    {"jahr": 2000, "text": "1. CO₂-Gesetz Schweiz", "y": 83, "ay": 40},
    {"jahr": 2005, "text": "Beginn Rückgang Erdöl", "y": 78, "ay": -40},
    {"jahr": 2010, "text": "Kohle-Rückgang", "y": 73, "ay": 40},
    {"jahr": 2011, "text": "Fukushima & Atomausstieg", "y": 68, "ay": -40},
    {"jahr": 2017, "text": "Energiestrategie 2050", "y": 63, "ay": 40},
    {"jahr": 2021, "text": "Blockiertes Stromabkommen", "y": 58, "ay": -40},
    {"jahr": 2022, "text": "Ukraine-Krieg / Gas-Krise", "y": 53, "ay": 40},
    {"jahr": 2023, "text": "Winterstromlücke (Debatte)", "y": 48, "ay": -40},
]

# Annotationen & Linien einfügen
for e in ereignisse:
    fig.add_vline(
        x=e["jahr"],
        line_width=2,
        line_dash="dash",
        line_color="gray"
    )
    fig.add_annotation(
        x=e["jahr"],
        y=e["y"],
        text=e["text"],
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        ax=0,
        ay=e["ay"],
        bgcolor="white",
        bordercolor="gray",
        borderwidth=1,
        font=dict(size=11)
    )


fig.show()


In [ ]:
df = pd.read_csv("/content/BINA-Projekt/Data/ogd115_gest_bilanz.csv")

In [ ]:
import pandas as pd
import plotly.express as px

# 1. Daten vorbereiten
df_import = df[df["Rubrik"] == "Import"].copy()
df_inland = df[df["Rubrik"] == "Inlandproduktion"].copy()

df_import_sum = df_import.groupby("Jahr")["TJ"].sum().reset_index().rename(columns={"TJ": "Import"})
df_inland_sum = df_inland.groupby("Jahr")["TJ"].sum().reset_index().rename(columns={"TJ": "Inlandproduktion"})

# 2. Zusammenführen
df_combined = df_import_sum.merge(df_inland_sum, on="Jahr")
df_combined["Total"] = df_combined["Import"] + df_combined["Inlandproduktion"]

# 3. Prozentanteile berechnen
df_combined["Import (%)"] = df_combined["Import"] / df_combined["Total"] * 100
df_combined["Inland (%)"] = df_combined["Inlandproduktion"] / df_combined["Total"] * 100

# 4. Long-Format für Area Chart
df_plot = df_combined.melt(id_vars="Jahr", value_vars=["Import (%)", "Inland (%)"],
                           var_name="Quelle", value_name="Anteil (%)")

# 5. Plotten
fig = px.area(
    df_plot,
    x="Jahr",
    y="Anteil (%)",
    color="Quelle",
    title="Importabhängigkeit der Schweiz beim Energieverbrauch (1980–2023)",
    labels={"Quelle": "Energieherkunft"},
    groupnorm="percent"
)

fig.update_layout(template="plotly_white", yaxis_range=[0, 100])
fig.show()


In [2]:
import panel as pn
import plotly.express as px
import pandas as pd

pn.extension("plotly")

# --- Plot 1 (dein erster Plot) ---
# (Hier die Originaldaten und dein Plotcode, leicht gekürzt)
df = pd.read_csv("/content/BINA-Projekt/Data/ogd115_gest_bilanz.csv")
df["Jahr"] = df["Jahr"].astype(int)
df["TJ"] = df["TJ"].astype(float)
df = df[df["Rubrik"] == "Import"]
importierte = ["Elektrizität", "Erdölprodukte","Gas","Kernbrennstoffe","Kohle","Rohöl","Uebrige erneuerbare Energien"]
df1 = df[df["Energietraeger"].isin(importierte)]

fig1 = px.line(
    df1,
    x="Jahr",
    y="TJ",
    color="Energietraeger",
    title="Bruttoverbrauch importierter Energieträger in der Schweiz (1980–2023)",
    labels={"TJ": "Energieverbrauch (TJ)", "Energietraeger": "Energieträger"},
    markers=True
)
fig1.update_layout(template="plotly_white")

ereignisse = [
    {"jahr": 1986, "text": "Tschernobyl / AKW-Debatte", "y": 98, "ay": -40},
    {"jahr": 1990, "text": "Kernenergie-Moratorium", "y": 92, "ay": 40},
    {"jahr": 1998, "text": "Strommarkt-Liberalisierung EU", "y": 88, "ay": -40},
    {"jahr": 2000, "text": "1. CO₂-Gesetz Schweiz", "y": 83, "ay": 40},
    {"jahr": 2005, "text": "Beginn Rückgang Erdöl", "y": 78, "ay": -40},
    {"jahr": 2010, "text": "Kohle-Rückgang", "y": 73, "ay": 40},
    {"jahr": 2011, "text": "Fukushima & Atomausstieg", "y": 68, "ay": -40},
    {"jahr": 2017, "text": "Energiestrategie 2050", "y": 63, "ay": 40},
    {"jahr": 2021, "text": "Blockiertes Stromabkommen", "y": 58, "ay": -40},
    {"jahr": 2022, "text": "Ukraine-Krieg / Gas-Krise", "y": 53, "ay": 60},
    {"jahr": 2023, "text": "Winterstromlücke (Debatte)", "y": 120, "ay": -60},
]
for e in ereignisse:
    fig1.add_vline(x=e["jahr"], line_width=2, line_dash="dash", line_color="gray")
    fig1.add_annotation(
        x=e["jahr"], y=e["y"], text=e["text"], showarrow=True,
        arrowhead=2, arrowsize=1, ax=0, ay=e["ay"],
        bgcolor="white", bordercolor="gray", borderwidth=1,
        font=dict(size=11)
    )

desc1 = """
Die Zeitreihe zeigt deutliche strukturelle Verschiebungen im Energieimportmix der Schweiz. Während Erdölprodukte historisch dominiert haben, ist ihr Verbrauch seit Mitte der 2000er rückläufig. Kernbrennstoffe hielten sich bis etwa 2010 stabil, bevor infolge von Fukushima der Atomausstieg eingeleitet wurde. Der Gasverbrauch zeigt moderate Volatilität, mit einem Anstieg nach 2015. Kohle wurde faktisch eliminiert. Elektrizität und übrige erneuerbare Energien bleiben mengenmäßig sekundär, aber strategisch zunehmend relevant. Externe Ereignisse wie die Tschernobyl-Katastrophe, Marktliberalisierungen, sowie geopolitische Krisen (Ukraine-Krieg, Gas-Krise) sind in den Daten sichtbar nachvollziehbar.
"""

# --- Plot 2 ---
df_grouped = df1.groupby(["Jahr", "Energietraeger"]).sum().reset_index()
df_total = df_grouped.groupby("Jahr")["TJ"].sum().reset_index().rename(columns={"TJ": "Total"})
df_merged = df_grouped.merge(df_total, on="Jahr")
df_merged["Anteil (%)"] = df_merged["TJ"] / df_merged["Total"] * 100

fig2 = px.area(
    df_merged,
    x="Jahr",
    y="Anteil (%)",
    color="Energietraeger",
    title="Relative Marktanteile importierter Energieträger (1980–2023)",
    groupnorm="percent"
)
fig2.update_layout(template="plotly_white")

ereignisse2 = [
    {"jahr": 1986, "text": "Tschernobyl / AKW-Debatte", "y": 98, "ay": -40},
    {"jahr": 1990, "text": "Kernenergie-Moratorium", "y": 92, "ay": 40},
    {"jahr": 1998, "text": "Strommarkt-Liberalisierung EU", "y": 88, "ay": -40},
    {"jahr": 2000, "text": "1. CO₂-Gesetz Schweiz", "y": 83, "ay": 40},
    {"jahr": 2005, "text": "Beginn Rückgang Erdöl", "y": 78, "ay": -40},
    {"jahr": 2010, "text": "Kohle-Rückgang", "y": 73, "ay": 40},
    {"jahr": 2011, "text": "Fukushima & Atomausstieg", "y": 68, "ay": -40},
    {"jahr": 2017, "text": "Energiestrategie 2050", "y": 63, "ay": 40},
    {"jahr": 2021, "text": "Blockiertes Stromabkommen", "y": 58, "ay": -40},
    {"jahr": 2022, "text": "Ukraine-Krieg / Gas-Krise", "y": 53, "ay": 40},
    {"jahr": 2023, "text": "Winterstromlücke (Debatte)", "y": 48, "ay": -40},
]
for e in ereignisse2:
    fig2.add_vline(x=e["jahr"], line_width=2, line_dash="dash", line_color="gray")
    fig2.add_annotation(
        x=e["jahr"], y=e["y"], text=e["text"], showarrow=True,
        arrowhead=2, arrowsize=1, ax=0, ay=e["ay"],
        bgcolor="white", bordercolor="gray", borderwidth=1,
        font=dict(size=11)
    )

desc2 = """
Diese Visualisierung legt den Fokus auf die prozentuale Zusammensetzung der Energieimporte. Erdölprodukte bleiben der dominante Energieträger, verlieren jedoch seit 2005 kontinuierlich Marktanteile. Der relative Anteil von Kernbrennstoffen sinkt ab 2011, während Gas leicht zulegt. Der vollständige Rückgang von Kohle und die marginale Rolle erneuerbarer Energieimporte markieren den Übergang hin zu einem diversifizierteren, aber weiterhin fossildominierten Energiemix. Politische Weichenstellungen – wie die Energiestrategie 2050 – zeigen spürbare Effekte auf die Importstruktur.
"""

# --- Plot 3 ---
df_import = df[df["Rubrik"] == "Import"].copy()
df_inland = df[df["Rubrik"] == "Inlandproduktion"].copy()

df_import_sum = df_import.groupby("Jahr")["TJ"].sum().reset_index().rename(columns={"TJ": "Import"})
df_inland_sum = df_inland.groupby("Jahr")["TJ"].sum().reset_index().rename(columns={"TJ": "Inlandproduktion"})

df_combined = df_import_sum.merge(df_inland_sum, on="Jahr")
df_combined["Total"] = df_combined["Import"] + df_combined["Inlandproduktion"]

df_combined["Import (%)"] = df_combined["Import"] / df_combined["Total"] * 100
df_combined["Inland (%)"] = df_combined["Inlandproduktion"] / df_combined["Total"] * 100

df_plot = df_combined.melt(id_vars="Jahr", value_vars=["Import (%)", "Inland (%)"],
                           var_name="Quelle", value_name="Anteil (%)")

fig3 = px.area(
    df_plot,
    x="Jahr",
    y="Anteil (%)",
    color="Quelle",
    title="Importabhängigkeit der Schweiz beim Energieverbrauch (1980–2023)",
    labels={"Quelle": "Energieherkunft"},
    groupnorm="percent"
)
fig3.update_layout(template="plotly_white", yaxis_range=[0, 100])

desc3 = """
Trotz politischer Bemühungen zur Förderung der inländischen Energieproduktion bleibt die Importabhängigkeit hoch. Seit über vier Jahrzehnten liegt der Importanteil konstant bei rund 80 %, mit einem leichten Rückgang auf ca. 75 % in den letzten Jahren. Die Inlandproduktion konnte ihren Anteil nur marginal steigern, was die strategische Verwundbarkeit der Schweiz in energiepolitischer Hinsicht unterstreicht. Für das Energiemanagement und die Versorgungssicherheit ergibt sich daraus ein klarer Handlungsbedarf.
"""

# --- Panel Layout ---

# Für jeden Plot + Text eine Spalte mit Plot oben, Text unten
pane1 = pn.Column(pn.pane.Plotly(fig1, sizing_mode="stretch_width"), pn.pane.Markdown(desc1))
pane2 = pn.Column(pn.pane.Plotly(fig2, sizing_mode="stretch_width"), pn.pane.Markdown(desc2))
pane3 = pn.Column(pn.pane.Plotly(fig3, sizing_mode="stretch_width"), pn.pane.Markdown(desc3))

# Alles untereinander anordnen
dashboard = pn.Column(pane1, pane2, pane3, sizing_mode="stretch_width")

dashboard.show()  # Oder dashboard.servable() im Notebook


Launching server at http://localhost:42683


In [4]:
import pandas as pd
import plotly.express as px
import panel as pn

pn.extension("plotly")

# --- Beispiel: Daten laden (du kannst hier deine CSV laden) ---
df = pd.read_csv("/content/BINA-Projekt/Data/ogd115_gest_bilanz.csv")

# --- Plot 1: Bruttoverbrauch importierter Energieträger ---
df["Jahr"] = df["Jahr"].astype(int)
df["TJ"] = df["TJ"].astype(float)
df1 = df[(df["Rubrik"] == "Import") & (df["Energietraeger"].isin(
    ["Elektrizität", "Erdölprodukte","Gas","Kernbrennstoffe","Kohle","Rohöl","Uebrige erneuerbare Energien"]))]

fig1 = px.line(
    df1,
    x="Jahr",
    y="TJ",
    color="Energietraeger",
    title="Bruttoverbrauch importierter Energieträger in der Schweiz (1980–2023)",
    labels={"TJ": "Energieverbrauch (TJ)", "Energietraeger": "Energieträger"},
    markers=True
)
fig1.update_layout(template="plotly_white")

desc1 = """
**Bruttoverbrauch importierter Energieträger (1980–2023)**

Die Zeitreihe zeigt deutliche strukturelle Verschiebungen im Energieimportmix der Schweiz.
Während Erdölprodukte historisch dominiert haben, ist ihr Verbrauch seit Mitte der 2000er rückläufig.
Kernbrennstoffe hielten sich bis etwa 2010 stabil, bevor infolge von Fukushima der Atomausstieg eingeleitet wurde.
Der Gasverbrauch zeigt moderate Volatilität, mit einem Anstieg nach 2015. Kohle wurde faktisch eliminiert.
Elektrizität und übrige erneuerbare Energien bleiben mengenmäßig sekundär, aber strategisch zunehmend relevant.
Externe Ereignisse wie die Tschernobyl-Katastrophe, Marktliberalisierungen sowie geopolitische Krisen sind sichtbar.
"""

# --- Plot 2: Relative Marktanteile importierter Energieträger ---
df2_grouped = df1.groupby(["Jahr", "Energietraeger"])["TJ"].sum().reset_index()
df2_total = df2_grouped.groupby("Jahr")["TJ"].sum().reset_index().rename(columns={"TJ": "Total"})
df2_merged = df2_grouped.merge(df2_total, on="Jahr")
df2_merged["Anteil (%)"] = df2_merged["TJ"] / df2_merged["Total"] * 100

fig2 = px.area(
    df2_merged,
    x="Jahr",
    y="Anteil (%)",
    color="Energietraeger",
    title="Relative Marktanteile importierter Energieträger (1980–2023)",
    groupnorm="percent"
)
fig2.update_layout(template="plotly_white")

desc2 = """
**Relative Marktanteile importierter Energieträger (1980–2023)**

Diese Visualisierung legt den Fokus auf die prozentuale Zusammensetzung der Energieimporte.
Erdölprodukte bleiben dominant, verlieren jedoch seit 2005 Marktanteile.
Kernbrennstoffe sinken ab 2011, Gas steigt leicht. Kohle verschwindet komplett.
Politische Weichenstellungen, wie die Energiestrategie 2050, zeigen Effekte auf die Importstruktur.
"""

# --- Plot 3: Importabhängigkeit beim Energieverbrauch ---
df_import = df[df["Rubrik"] == "Import"].groupby("Jahr")["TJ"].sum().reset_index().rename(columns={"TJ": "Import"})
df_inland = df[df["Rubrik"] == "Inlandproduktion"].groupby("Jahr")["TJ"].sum().reset_index().rename(columns={"TJ": "Inlandproduktion"})
df3 = df_import.merge(df_inland, on="Jahr")
df3["Total"] = df3["Import"] + df3["Inlandproduktion"]
df3["Import (%)"] = df3["Import"] / df3["Total"] * 100
df3["Inland (%)"] = df3["Inlandproduktion"] / df3["Total"] * 100
df3_long = df3.melt(id_vars="Jahr", value_vars=["Import (%)", "Inland (%)"], var_name="Quelle", value_name="Anteil (%)")

fig3 = px.area(
    df3_long,
    x="Jahr",
    y="Anteil (%)",
    color="Quelle",
    title="Importabhängigkeit der Schweiz beim Energieverbrauch (1980–2023)",
    groupnorm="percent",
    labels={"Quelle": "Energieherkunft"}
)
fig3.update_layout(template="plotly_white", yaxis_range=[0, 100])

desc3 = """
**Importabhängigkeit beim Energieverbrauch (1980–2023)**

Trotz Bemühungen zur Förderung der inländischen Energieproduktion bleibt die Importabhängigkeit hoch.
Seit Jahrzehnten liegt der Importanteil konstant bei ca. 80 %, mit leichtem Rückgang auf 75 % zuletzt.
Die Inlandproduktion konnte nur marginal zulegen. Strategische Verwundbarkeit bleibt bestehen.
"""

# --- Panel Layout mit Tabs ---
pane1 = pn.Column(pn.pane.Plotly(fig1, sizing_mode="stretch_width"), pn.pane.Markdown(desc1))
pane2 = pn.Column(pn.pane.Plotly(fig2, sizing_mode="stretch_width"), pn.pane.Markdown(desc2))
pane3 = pn.Column(pn.pane.Plotly(fig3, sizing_mode="stretch_width"), pn.pane.Markdown(desc3))

tabs = pn.Tabs(
    ("Bruttoverbrauch importierter Energieträger", pane1),
    ("Relative Marktanteile importierter Energieträger", pane2),
    ("Importabhängigkeit beim Energieverbrauch", pane3),
    sizing_mode="stretch_width"
)

tabs  # Ausgabe in Jupyter Notebook - zeigt interaktive Tabs inline


Tabs(sizing_mode='stretch_width')
    [0] Column
        [0] Plotly(Figure, sizing_mode='stretch_width')
        [1] Markdown(str)
    [1] Column
        [0] Plotly(Figure, sizing_mode='stretch_width')
        [1] Markdown(str)
    [2] Column
        [0] Plotly(Figure, sizing_mode='stretch_width')
        [1] Markdown(str)

#Analyse

Im Folgenden werden die Resultate einer Analyse präsentiert, die sich mit der Entwicklung importierter Energieträger in der Schweiz befasst.
Eine Analyse des langfristigen Trends des Bruttoverbrauchs importierter Energieträger in der Schweiz zeigt signifikante strukturelle Veränderungen, die auf technologische Entwicklungen, energiepolitische Entscheidungen und geopolitische Ereignisse zurückzuführen sind.

Über einen Zeitraum von mehreren Jahrzehnten waren Erdölprodukte für den Schweizer Energiemix von herausragender Bedeutung. Ihr Einfluss war insbesondere im Verkehrssektor sowie im Bereich der Gebäudeheizung signifikant. Seit etwa 2005 lässt sich jedoch ein kontinuierlicher Rückgang im Verbrauch verzeichnen. Diese Entwicklung ist in erster Linie auf Effizienzsteigerungen – beispielsweise durch verbesserte Motorentechnologie oder optimierte Heizsysteme – sowie auf den schrittweisen Umstieg auf alternative Antriebstechnologien wie Elektromobilität zurückzuführen. Zusätzlich wirkten regulatorische Massnahmen, wie etwa die CO2-Abgabe oder verbindliche Klimaziele, dämpfend auf den Verbrauch.

Auch beim Erdgas zeigt sich eine differenzierte Entwicklung. Obwohl der Verbrauch über einen längeren Zeitraum relativ stabil blieb, zeigen die Daten seit etwa 2010 eine zunehmende Volatilität. Der signifikante Rückgang ab dem Jahr 2022 ist insbesondere vor dem Hintergrund der geopolitischen Instabilität in Europa – insbesondere dem Krieg in der Ukraine und der daraus resultierenden Energiekrise – zu interpretieren. Infolge dieser Unsicherheiten wurde die strategische Verwundbarkeit der Schweiz in Bezug auf Gasimporte evident. Als Reaktion auf diese Entwicklung begannen Industrie und Haushalte verstärkt, Gas durch andere Energieformen zu substituieren. Dies dürfte mittelfristig zu einer weiteren Reduktion des Gasverbrauchs führen.
Es ist von essentieller Bedeutung, dass die Analyse der vorliegenden Daten mit äusserster Sorgfalt erfolgt.

Demgegenüber weist Elektrizität ein deutlich schwankenderes Verbrauchsprofil auf, das in hohem Masse von saisonalen Einflüssen, der inländischen Produktionskapazität (insbesondere Wasserkraft) und dem Importbedarf abhängt. Elektrischer Strom nimmt eine zunehmend zentrale Rolle in der Dekarbonisierungsstrategie der Schweiz ein. Dies manifestiert sich einerseits im Ersatz fossiler Energieträger in der Mobilität und andererseits im verstärkten Einsatz von Wärmepumpen im Gebäudebereich. Diese Entwicklung ist jedoch mit Herausforderungen verbunden. Die sogenannte Winterstromlücke, die durch die unzureichende einheimische Produktion in den Wintermonaten entsteht, führt zu temporären Importen. Diese sind wiederum mit Versorgungsrisiken und Preisschwankungen verbunden, was die Notwendigkeit eines Ausbaus von Speicherlösungen und eines diversifizierten Energieportfolios unterstreicht.

Schliesslich lässt sich bei den Kohleprodukten ein nahezu vollständiger Bedeutungsverlust beobachten. Seit den 1990er Jahren ist ein kontinuierlicher Rückgang des Verbrauchs zu verzeichnen, sodass sie heute im schweizerischen Energiesystem eine marginale Rolle spielen. Dieser Rückgang ist Ausdruck eines erfolgreichen Strukturwandels, der durch politische Rahmenbedingungen, ökologische Zielsetzungen und die mangelnde wirtschaftliche Attraktivität des kohlebasierten Energieverbrauchs bedingt ist.

Die Daten weisen insgesamt eine signifikante Verschiebung hin zu elektrischer Energie, wobei diese zwar klimafreundlicher ist, jedoch weiterhin importabhängig bleibt. Die Schweiz sieht sich folglich mit der strategischen Herausforderung konfrontiert, ihre Energieversorgung breiter abzustützen, Importe gezielter zu diversifizieren und die eigene Produktionskapazität nachhaltig auszubauen.



#Ursachen

Im Folgenden werden die identifizierten Trendbrüche und deren Ursachen dargelegt.
Die Analyse der Gesamtenergiestatistik offenbart signifikante Trendbrüche im Energieimportverhalten der Schweiz, die sich über spezifische Zeiträume hinweg identifizieren lassen. Die vorliegenden Veränderungen stehen in einem engen Zusammenhang mit technologischen Entwicklungen, energiepolitischen Massnahmen sowie geopolitischen Rahmenbedingungen.

Ein erster signifikanter Trend manifestiert sich ab dem Jahr 2005: Seither ist ein kontinuierlicher Rückgang des Verbrauchs von Erdölprodukten zu beobachten. Diese Entwicklung ist primär auf Fortschritte in der Energieeffizienz, insbesondere im Automobil- und Heizsektor, sowie auf politische Massnahmen zur Dekarbonisierung zurückzuführen. Darüber hinaus beeinflusst die zunehmende Verbreitung der Elektromobilität die Nachfrage nach fossilen Brennstoffen.

Ein weiterer Wendepunkt lässt sich seit dem Jahr 2010 feststellen, als der Verbrauch von Kohleprodukten signifikant rückläufig war. Dieser Rückgang ist sowohl Ausdruck energie- und klimapolitischer Zielsetzungen als auch Resultat einer systematischen Substitution kohlebasierter Prozesse, insbesondere in der Industrie. Die geringe gesellschaftliche und politische Akzeptanz von Kohle sowie ihre vergleichsweise schlechte Umweltbilanz haben ihre Rolle im Energiemix marginalisiert.

Ein besonders aktueller und einschneidender Bruch ereignete sich ab dem Jahr 2022, als der Verbrauch von Gas deutlich zurückging. Die Ursachen hierfür sind in der geopolitischen Instabilität Europas zu verorten, konkret im Krieg in der Ukraine und der damit verbundenen Gasversorgungskrise. Diese Ereignisse haben die Verwundbarkeit der Schweiz im Bereich fossiler Energieimporte verdeutlicht und zu einer kurzfristigen Verhaltensanpassung auf politischer wie wirtschaftlicher Ebene geführt.

Schliesslich sind über den gesamten Zeitraum hinweg starke Schwankungen beim Elektrizitätsimport feststellbar. Diese Entwicklung lässt sich nicht auf einen einzelnen Trendbruch zurückführen, sondern resultiert aus strukturellen Rahmenbedingungen. Die saisonale Produktion von Wasserkraft, in Kombination mit der sogenannten Winterstromlücke, resultiert in einer verstärkten Importabhängigkeit der Schweiz während der Wintermonate. Diese Schwankungen sind Ausdruck eines Spannungsfeldes zwischen inländischer Produktionskapazität, wetterabhängiger Erzeugung und struktureller Importnotwendigkeit.

Die identifizierten Brüche können als Ausdruck eines sich wandelnden Energiesystems interpretiert werden, das zunehmend von Nachhaltigkeitszielen, technologischen Transformationen und äusseren Schocks geprägt ist. Diese Erkenntnisse bilden eine zentrale Grundlage für die energiepolitische Steuerung und die zukünftige Ausrichtung der Schweizer Energieversorgung im Rahmen der Energiestrategie 2050.



#Handlungsempfehlungen

Im Folgenden werden Handlungsempfehlungen dargelegt, die darauf abzielen, die Versorgungssicherheit zu stärken und die Energiestrategie 2050 umzusetzen.
Aus der Analyse der Entwicklungen importierter Energieträger ergeben sich mehrere konkrete Handlungsfelder, die zur langfristigen Stabilisierung der Energieversorgung und zur erfolgreichen Umsetzung der Energiestrategie 2050 beitragen können.

Ein zentraler Ansatzpunkt liegt im Ausbau der inländischen Stromproduktion. Die vorliegende Untersuchung kommt zu dem Schluss, dass die zunehmende Elektrifizierung des Verkehrs- und Wärmesektors nachhaltig nur dann getragen werden kann, wenn gleichzeitig die Importabhängigkeit in den kritischen Wintermonaten reduziert wird. Zu diesem Zweck sind Photovoltaik, Windkraft und Wasserkraft gezielt zu fördern. Der Ausbau von Speicherlösungen – insbesondere saisonalen Speichern – erweist sich als eine entscheidende Massnahme, um Lastspitzen abzufedern und Versorgungslücken zu vermeiden.

Zur Reduktion geopolitischer Risiken sollten Importrisiken aktiv minimiert werden. Dies umfasst sowohl die Diversifikation der Bezugsquellen für Strom und Gas als auch den gezielten Aufbau redundanter Infrastrukturen, die im Krisenfall alternative Versorgungswege ermöglichen. Dies ist insbesondere im Gassektor von strategischer Relevanz, wie die Entwicklungen seit 2022 evident belegen.

Darüber hinaus wird dem Abschluss langfristiger Energiepartnerschaften, beispielsweise in Form eines stabilen Stromabkommens mit der Europäischen Union, eine hohe Priorität beigemessen. Derartige Übereinkünfte können einen Beitrag zur Absicherung grenzüberschreitender Energieflüsse leisten und einen geordneten Zugang zum europäischen Strommarkt ermöglichen. Dieser Aspekt kann insbesondere in Zeiten hoher Netzbelastung von entscheidender Bedeutung sein.

Um eine optimale Vorbereitung auf potenzielle zukünftige Versorgungsschwankungen zu gewährleisten, ist eine Erweiterung des energiestatistischen Monitorings unabdingbar. Ein Frühwarnsystem, das Engpässe, Marktverwerfungen oder geopolitische Risiken frühzeitig erkennt, kann sowohl politische als auch betriebliche Akteure bei der Reaktionsplanung unterstützen.

Schliesslich ist auch die transparente Kommunikation energiepolitischer Entwicklungen von zentraler Bedeutung. Der Einsatz interaktiver Dashboards, offener Datenplattformen und zielgruppengerechter Datenvisualisierungen kann dazu beitragen, komplexe Zusammenhänge verständlich zu vermitteln und das Vertrauen in energiepolitische Entscheidungen zu stärken. Dies unterstützt nicht nur die politische Legitimation, sondern fördert auch das Engagement von Wirtschaft und Zivilgesellschaft in der Umsetzung der Energiewende.

#Fazit

Die vorliegende Untersuchung gelangt zu folgendem Schluss:
Die Analyse historischer Entwicklungen importierter Energieträger demonstriert, dass die Schweiz strukturell auf Energieimporte angewiesen ist, sowohl im fossilen als auch im elektrischen Sektor. Historisch betrachtet waren Erdöl und Kohle die dominierenden Energiequellen. Im Zuge der Dekarbonisierung und technologischen Transformation hat sich die Abhängigkeit jedoch zugunsten von Gas und Strom verschoben.

Diese Verschiebung geht mit neuen Herausforderungen einher, die im Folgenden diskutiert werden sollen. Zunächst ist die Importabhängigkeit bei Strom in den Wintermonaten zu nennen, die die Verletzlichkeit des aktuellen Systems verdeutlicht. Auch die volatilen Gasimporte im Kontext geopolitischer Krisen sind in diesem Zusammenhang zu betrachten.

Für eine langfristig krisenfeste, nachhaltige und wirtschaftlich tragfähige Energieversorgung der Schweiz ergeben sich daraus vier zentrale Handlungsschwerpunkte:

1. Die Stärkung der inländischen Energieproduktion sollte insbesondere durch den Ausbau erneuerbarer Quellen wie Photovoltaik, Wind- und Wasserkraft erfolgen.

2. Die Integration intelligenter Speicherlösungen stellt eine Möglichkeit dar, saisonale und tageszeitliche Versorgungslücken zu überbrücken.

3. Die Reduktion politischer Abhängigkeiten kann durch Diversifikation der Importkanäle und strategische Partnerschaften erreicht werden.

4. Die Bereitstellung von transparenten und zugänglichen Datengrundlagen ist von entscheidender Bedeutung, um evidenzbasierte Entscheidungen zu ermöglichen und das Vertrauen in energiepolitische Massnahmen zu stärken.

Die Gewährleistung der Energieversorgung stellt eine Herausforderung dar, die nicht allein auf technischer Ebene zu bewältigen ist, sondern auch politische und gesellschaftliche Aspekte umfasst. Die vorliegende Analyse bildet eine faktenbasierte Grundlage für die Weiterentwicklung der Schweizer Energiepolitik im Sinne der Energiestrategie 2050.

